In [72]:
import requests
import pandas as pd
import us

In [73]:
# Define base URL and API key
base_url = "https://api.census.gov/data/timeseries/poverty/saipe"
API_KEY = "a9ac3d6b748deca8c30be3b51da20843b0f8adbb"

# Function to fetch data for a specific year, state, and county

def fetch_data(year, state_code, county_code, get_metric='SAEMHI_PT'):

    params = {
        'get': get_metric, # see https://api.census.gov/data/timeseries/poverty/saipe/variables.html
        'for': f'county:{county_code}',
        'in': f'state:{state_code}',
        'time': str(year),
        'key': API_KEY
    }
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Error for year {year}, {state_code}{county_code}: {response.text}")
        return None

    try:
        data = response.json()[1][0]
        # print(f"Data for year {year}, {state_code}{county_code}: {data}")  # This will print the data structure
        return data
    except JSONDecodeError:
        print(f"JSON decode error for year {year}, {county_code}. Response content: {response.text}")
        return None




# List of state codes
# Note: This list contains all 50 US state codes. Adjust if necessary.
state_fips = [state.fips for state in us.STATES]

url_FIPS = "https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt"
df_county_fips = pd.read_csv(url_FIPS, header=None, dtype=str)
df_county_fips.columns = ["State", "State FIPS", "County FIPS", "County Name", "Class Code"]
df_county_fips

# Fetch the data
years = range(2001, 2023)  # 2019 to 2022 inclusive
# initialize df with columns for geoFIPS and all years
df = pd.DataFrame(columns=['GeoFIPS'] + [str(year) for year in years])
df.set_index('GeoFIPS', inplace=True)

for year in years:
    print(f"Fetching data for year {year}")
    for sci in range(len(df_county_fips)):
        state_code = df_county_fips.loc[sci, "State FIPS"]
        county_code = df_county_fips.loc[sci, "County FIPS"]
        data = fetch_data(year, state_code, county_code)
        if data:
            GeoFIPS = state_code + county_code
            # If GeoFIPS doesn't exist in the df, add it. Else, update the year column.
            if GeoFIPS not in df.index:
                df.loc[GeoFIPS] = [None] * len(df.columns)

            df.loc[GeoFIPS, str(year)] = data


print(df)

Data for year 2001, 01001: 42183
Data for year 2001, 01003: 40406
Data for year 2001, 01005: 25480
Data for year 2001, 01007: 31074
Data for year 2001, 01009: 36175
Data for year 2001, 01011: 20367
Data for year 2001, 01013: 25284
Data for year 2001, 01015: 31849
Data for year 2001, 01017: 29824
Data for year 2001, 01019: 30738
Data for year 2001, 01021: 32883
Data for year 2001, 01023: 25878
Data for year 2001, 01025: 28010
Data for year 2001, 01027: 28289
Data for year 2001, 01029: 31125
Data for year 2001, 01031: 33916
Data for year 2001, 01033: 32249
Data for year 2001, 01035: 23177


KeyboardInterrupt: 

In [71]:
# save to csv
# df.to_csv("../../data/processed/median_personal_income.csv")
df

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
GeoFIPS,,,,,,,,,,,,,,,,,,,,,
01001,42183,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
01003,40406,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
01005,25480,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
01007,31074,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
01009,36175,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05025,33511,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
05027,28385,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
05029,31166,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
